In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [4]:
PROJ_DIR = Path.home()/"Sync"/"KiddLab"/"MSM"
STIM_DIR = PROJ_DIR/"assets"/"stimuli"
STIM_DATABASE = pd.read_csv(STIM_DIR/"stimulus_database.csv")

In [5]:
STIM_DATABASE

,stim_type,alternation_rate,target_talker,target_sentence,init_target_position,init_target_moving_right,masker_talker,masker_sentence,init_masker_position,init_masker_moving_right
0,SOS,0.5,1M,Sue gave two cheap socks or Bob sold three gre...,-90,True,2M,Bob held nine small gloves where Jill held six...,90,False
1,SOS,0.5,11M,Sue sold four small gloves when Bob found thre...,-90,True,9M,Mike found two big bags or Sam took nine new hats,90,False
2,SOS,0.5,7M,Sue held two cheap cards if Jill held two chea...,-90,True,1M,Jane gave six green toys while Jane found nine...,90,False
3,SOS,0.5,7M,Sue gave six cheap hats if Jane sold nine red ...,-90,True,1M,Mike held five green gloves while Bob saw four...,90,False
4,SOS,0.5,1M,Sue bought four hot toys when Bob sold five ho...,90,True,10M,Sam gave three green hats while Sam gave three...,-90,False
...,...,...,...,...,...,...,...,...,...,...
65,SOS,10.0,9M,Sue bought ten cheap hats and Lynn took eight ...,-90,True,1M,Pat gave two hot bags but Pat lost four green ...,90,False
66,SOS,10.0,1M,Sue bought four small toys then Jill held two ...,-90,True,8M,Lynn took eight big cards and Sam took nine bi...,90,False
67,SOS,10.0,10M,Sue sold eight red shoes and Mike gave ten gre...,90,True,7M,Jill bought five new gloves if Pat held five c...,-90,False
68,SOS,10.0,4M,Sue saw ten new bags if Mike sold two red gloves,-90,True,11M,Bob lost four big pens when Jill saw eight sma...,90,False


In [6]:
STIM_DATABASE.append( \
            {"stim_type": "SOS",
             "alternation_rate": 12,
             "target_talker": "D",
             "target_sentence": "S",
             "init_target_position": 90,
             "init_target_moving_right": True,
             "masker_talker": "M",
             "masker_sentence": "R",
             "init_masker_position": -90,
             "init_masker_moving_right": False},
             ignore_index=True)


,stim_type,alternation_rate,target_talker,target_sentence,init_target_position,init_target_moving_right,masker_talker,masker_sentence,init_masker_position,init_masker_moving_right
0,SOS,0.5,1M,Sue gave two cheap socks or Bob sold three gre...,-90,True,2M,Bob held nine small gloves where Jill held six...,90,False
1,SOS,0.5,11M,Sue sold four small gloves when Bob found thre...,-90,True,9M,Mike found two big bags or Sam took nine new hats,90,False
2,SOS,0.5,7M,Sue held two cheap cards if Jill held two chea...,-90,True,1M,Jane gave six green toys while Jane found nine...,90,False
3,SOS,0.5,7M,Sue gave six cheap hats if Jane sold nine red ...,-90,True,1M,Mike held five green gloves while Bob saw four...,90,False
4,SOS,0.5,1M,Sue bought four hot toys when Bob sold five ho...,90,True,10M,Sam gave three green hats while Sam gave three...,-90,False
...,...,...,...,...,...,...,...,...,...,...
66,SOS,10.0,1M,Sue bought four small toys then Jill held two ...,-90,True,8M,Lynn took eight big cards and Sam took nine bi...,90,False
67,SOS,10.0,10M,Sue sold eight red shoes and Mike gave ten gre...,90,True,7M,Jill bought five new gloves if Pat held five c...,-90,False
68,SOS,10.0,4M,Sue saw ten new bags if Mike sold two red gloves,-90,True,11M,Bob lost four big pens when Jill saw eight sma...,90,False
69,SOS,10.0,9M,Sue lost three new shoes when Pat lost five gr...,-90,True,2M,Lynn bought ten big socks where Jill gave two ...,90,False


In [7]:
STIM_DATABASE.to_csv("test.csv")